# Añadir y eliminar datos

## Acerca de los datos
En este cuaderno trabajaremos con datos de terremotos del 18 de septiembre de 2018 al 13 de octubre de 2018 (obtenidos del US Geological Survey (USGS) mediante la [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Configuración
Estaremos trabajando con el archivo `data/earthquakes.csv` nuevamente, por lo que necesitamos manejar nuestras importaciones y leerlo.

In [9]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Creación de nuevos datos
### Añadir nuevas columnas
Las nuevas columnas se añaden a la derecha de las columnas originales y pueden ser un único valor, que será **difundido** a lo largo de las filas del marco de datos:

In [10]:
df['source'] = 'USGS API'
df.head()

,alert,mag,magType,place,time,title,tsunami,source
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API


...o una máscara booleana:

In [11]:
df['mag_negative'] = df.mag < 0
df.head()

,alert,mag,magType,place,time,title,tsunami,source,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API,False


#### Añadir la columna `parsed_place`
Tenemos un problema de reconocimiento de entidades con la columna `place`. Hay varias entidades que tienen varios nombres en los datos (por ejemplo, CA y California, NV y Nevada).

In [12]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

Reemplazar partes de los nombres `place` para adaptarlos a nuestras necesidades:

In [13]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True # eliminar el "of"
).str.replace(
    'the ', '' # eliminar "the"
).str.replace(
    r'CA$', 'California', regex=True # Corregir California
).str.replace(
    r'NV$', 'Nevada', regex=True # Corregir Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True # Corregir Mexico
).str.replace(
    r' region$', '', regex=True # cortar las terminaciones con "región"
).str.replace(
    'northern ', '' # eliminar "northern"
).str.replace(
    'Fiji Islands', 'Fiji' # Alinear las plazas de Fiji
).str.replace(
    r'^.*, ', '', regex=True # eliminar cualquier otra cosa extraña desde el principio
).str.strip() # eliminar los espacios sobrantes

Ahora podemos utilizar un solo nombre para obtener todos los terremotos de ese lugar (aunque esto todavía no es perfecto):

In [14]:
df.parsed_place.sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Ascension Island', 'Australia', 'Azerbaijan', 'Balleny Islands',
       'Barbuda', 'Bolivia', 'British Virgin Islands', 'Burma',
       'California', 'Canada', 'Carlsberg Ridge',
       'Central East Pacific Rise', 'Central Mid-Atlantic Ridge', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'El Salvador',
       'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala', 'Haiti',
       'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indian Ocean Triple Junction', 'Indonesia', 'Iran', 'Iraq',
       'Italy', 'Jamaica', 'Japan', 'Kansas', 'Kentucky',
       'Kermadec Islands', 'Kuril Islands', 'Kyrgyzstan', 'Martinique',
       'Mauritius', 'Mayotte', 'Mexico', 'Mid-Indian Ridge', 'Missouri',
       'Montana', 'Nevada', 'New Caledonia', 'New Hampshire',
       'New Mexico', 'New Zealand', 'Nicaragua', 'North Carolina',


#### Utilización del método `assign()` para crear columnas
Para crear muchas columnas a la vez o actualizar columnas existentes, podemos utilizar `assign()`:

In [15]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False


Con el uso de funciones `lambda`, el método `assign()` se vuelve aún más potente. **Las funciones lambda** son funciones anónimas que suelen definirse en una sola línea y para un solo uso. El método `assign()` pasa todo el marco de datos a la función `lambda` como `x`; desde ahí, podemos seleccionar las columnas `in_ca` y `in_alaska`, que se están creando en esa misma llamada a `assign()`. Aquí, utilizamos una función `lambda` para crear una nueva columna, `neither`, que indica si el terremoto no se produjo ni en Alaska ni en California:

In [16]:
df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska,neither
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False,True
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False,True
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True,False
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True,False
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False,False


#### Concatenación
Supongamos que trabajamos con dos marcos de datos distintos, uno con terremotos acompañados de tsunamis y otro con terremotos sin tsunamis. Si quisiéramos ver los terremotos en su conjunto, tendríamos que concatenar los marcos de datos en uno solo:

In [17]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

((61, 10), (9271, 10))

Concatenar a lo largo del eje de filas (`axis=0`) equivale a añadir hasta el final. Al concatenar los terremotos con tsunami y sin tsunami, obtenemos el conjunto completo de datos de terremotos:

In [18]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

Observe que el resultado anterior es equivalente a ejecutar el método `append()` del marco de datos:

In [19]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

Hemos estado trabajando con un subconjunto de las columnas del fichero CSV, pero supongamos que ahora queremos obtener algunas de las columnas que ignoramos cuando leímos los datos. Como hemos añadido nuevas columnas en este cuaderno, no querremos leer el fichero y volver a realizar esas operaciones. En su lugar, concatenaremos a lo largo de las columnas (`axis=1`) para volver a añadir lo que nos falta:

In [20]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389202,",-480.0


Pero fíjate en lo que ocurre si el índice no está alineado:

In [21]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.4 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.3 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1539475168010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389218,",-480.0
1539475129610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389202,",-480.0


Si el índice no está alineado, podemos alinearlo antes de intentar la concatenación, de la que hablaremos en el capítulo 3.

Digamos que queremos unir los marcos de datos `tsunami` y `no_tsunami`, pero el marco de datos `no_tsunami` tiene una columna adicional. El parámetro `join` especifica cómo manejar cualquier solapamiento en los nombres de las columnas (cuando se añaden a la parte inferior) o en los nombres de las filas (cuando se concatenan a la izquierda/derecha). Por defecto, este parámetro es `outer`, por lo que conservamos todo; sin embargo, si utilizamos `inner`, sólo conservaremos lo que esté en común:

In [22]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


Además, utilizamos `ignore_index`, ya que el índice no significa nada para nosotros aquí. Esto nos da valores secuenciales en lugar de lo que teníamos en el resultado anterior:

In [23]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
0,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
1,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
2,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
3,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


## Borrado de datos no deseados
Las columnas pueden borrarse utilizando la sintaxis de diccionario con `del`:

In [24]:
del df['source']
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'mag_negative', 'parsed_place'],
      dtype='object')

Si no sabemos si la columna existe, debemos utilizar un bloque `try`/`except`:

In [25]:
try:
    del df['source']
except KeyError:
    # handle the error here
    print('not there anymore')

not there anymore


También podemos utilizar `pop()`. Esto nos permitirá utilizar las series que eliminemos más tarde. Tenga en cuenta que habrá un error si la clave no existe, por lo que también podemos utilizar un `try`/`except` aquí:

In [26]:
mag_negative = df.pop('mag_negative')
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'parsed_place'],
      dtype='object')

Fíjate que ahora tenemos una máscara en `mag_negative`:

In [27]:
mag_negative.value_counts()

mag_negative
False    8841
True      491
Name: count, dtype: int64

Ahora, podemos utilizar `mag_negative` para filtrar nuestros datos:

In [28]:
df[mag_negative].head()

,alert,mag,magType,place,time,title,tsunami,parsed_place
39,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539458844506,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
49,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539455017464,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
135,NaN,-0.40,ml,"10km SSE of Beatty, Nevada",1539422175717,"M -0.4 - 10km SSE of Beatty, Nevada",0,Nevada
161,NaN,-0.02,md,"20km SSE of Ronan, Montana",1539412475360,"M -0.0 - 20km SSE of Ronan, Montana",0,Montana
198,NaN,-0.20,ml,"60km N of Pahrump, Nevada",1539398340822,"M -0.2 - 60km N of Pahrump, Nevada",0,Nevada


### Usando el método `drop()`.
Podemos eliminar filas pasando una lista de índices al método `drop()`. Observa en el siguiente ejemplo que cuando pedimos las 2 primeras filas con `head()` obtenemos la 3ª y 4ª filas porque hemos eliminado las 2 primeras originales con `drop([0, 1])`:

In [29]:
df.drop([0, 1]).head(2)

,alert,mag,magType,place,time,title,tsunami,parsed_place
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,California


El método `drop()` elimina por defecto a lo largo del eje de filas. Si pasamos una lista de columnas con el argumento `columns`, podemos eliminar columnas:

In [30]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


También tenemos la opción de utilizar `axis=1`:

In [31]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

True

Por defecto, `drop()`, junto con la mayoría de los métodos `DataFrame`, devolverá un nuevo objeto `DataFrame`. Si sólo queremos cambiar el objeto con el que estamos trabajando, podemos pasarle `inplace=True`. Esto debe usarse con cuidado:

In [32]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="text-align: left;">
        <a href="./5-subconjunto_data.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="text-align: center;">
        <a href="../solutions/ch_02/solutions.ipynb">
            <button>Soluciones</button>
        </a>
    </div>
    <div style="text-align: right;">
        <a href="../ch_03/1-ancho_vs_largo.ipynb">
            <button>Capitulo 3 &#8594;</button>
        </a>
    </div>
</div>

<hr>
